## Install necessary libraries

In [10]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
!pip uninstall pinecone-client -y -q
!pip install sentence_transformers -q
!pip install pinecone -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.6 MB/s eta 0:00:00


## Load data

In [11]:
cricket_news = """
The T20 World Cup 2024 is in full swing, bringing excitement and drama to cricket fans worldwide.
India's team, captained by Rohit Sharma, is preparing for a crucial match against Ireland, with standout player Jasprit Bumrah expected to play a pivotal role in their campaign.
The tournament has already seen controversy, particularly concerning the pitch conditions at Nassau County International Cricket Stadium in New York, which came under fire after a low-scoring game between Sri Lanka and South Africa.
"""

football_news = """
The world of football is buzzing with excitement as major tournaments and league matches continue to captivate fans globally.
In the UEFA Champions League, the semi-final matchups have been set, with defending champions Real Madrid set to face Manchester City, while Bayern Munich will take on Paris Saint-Germain.
Both ties promise thrilling encounters, featuring some of the best talents in world football.
"""

election_news = """
As election season heats up, the latest developments reveal a highly competitive atmosphere across several key races.
The presidential election has seen intense campaigning from all major candidates, with recent polls indicating a tight race.
Incumbent President Jane Doe is seeking re-election on a platform of economic stability and healthcare reform, while her main rival, Senator John Smith, focuses on education and climate change initiatives."""


ai_revolution_news = """
The AI revolution continues to transform industries and reshape the global economy.
Significant advancements in artificial intelligence have led to breakthroughs in healthcare, with AI-driven diagnostics improving patient outcomes and reducing costs.
Autonomous systems are becoming increasingly prevalent in logistics and transportation, enhancing efficiency and safety."""

## Perform embeddings on data

In [12]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-mpnet-base-v2")

In [6]:
embeddings = embedding_model.encode([cricket_news, football_news, election_news, ai_revolution_news])

In [7]:
embeddings

array([[-0.02901842,  0.0192444 , -0.0181424 , ...,  0.00644327,
        -0.01740812, -0.01381658],
       [-0.00384662, -0.07271519, -0.00284145, ..., -0.02027755,
         0.02123847, -0.03015987],
       [-0.02962372,  0.05711373,  0.01119961, ...,  0.0131924 ,
         0.02634867,  0.01807423],
       [-0.01667612,  0.05068192, -0.05662728, ..., -0.00878626,
        -0.02318501, -0.04949613]], dtype=float32)

In [13]:
len(embeddings[0])

768

## Initiate Pinecone

In [ ]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key="API KEY") # add your api key here
spec = ServerlessSpec(cloud='aws', region='us-east-1')

## Create Index

In [16]:
pc.create_index("example-index2", dimension=768, metric="cosine", spec=spec)

{
    "name": "example-index2",
    "metric": "cosine",
    "host": "example-index2-9n7chcf.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "region": "us-east-1",
            "cloud": "aws",
            "read_capacity": {
                "mode": "OnDemand",
                "status": {
                    "state": "Ready",
                    "current_shards": null,
                    "current_replicas": null
                }
            }
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 768,
    "deletion_protection": "disabled",
    "tags": null,
    "_response_info": {
        "raw_headers": {
            "content-type": "application/json",
            "access-control-allow-origin": "*",
            "vary": "origin,access-control-request-method,access-control-request-headers",
            "access-control-expose-headers": "*",
            "x-pinecone-api-version": "

In [17]:
pc.list_indexes()

[
    {
        "name": "example-index2",
        "metric": "cosine",
        "host": "example-index2-9n7chcf.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "region": "us-east-1",
                "cloud": "aws",
                "read_capacity": {
                    "mode": "OnDemand",
                    "status": {
                        "state": "Ready",
                        "current_shards": null,
                        "current_replicas": null
                    }
                }
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 768,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "example-index",
        "metric": "cosine",
        "host": "example-index-9n7chcf.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "region": "u

## Use Index

In [18]:
index = pc.Index("example-index2")

In [19]:
index.describe_index_stats()

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '150',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 10 Dec 2025 11:42:59 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '43',
                                    'x-pinecone-request-id': '6980119734734166844',
                                    'x-pinecone-request-latency-ms': '42'}},
 'dimension': 768,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'storageFullness': 0.0,
 'total_vector_count': 0,
 'vector_type': 'dense'}

## Add data to Pinecone Index

In [20]:
index.upsert([
    {"id":"id1", "values":embeddings[0], "metadata":{'source': 'cricket'}},
    {"id":"id2", "values":embeddings[1], "metadata":{'source': 'football'}},
    {"id":"id3", "values":embeddings[2], "metadata":{'source': 'election'}},
    {"id":"id4", "values":embeddings[3], "metadata":{'source': 'ai_revolution'}}
])

UpsertResponse(upserted_count=4, _response_info={'raw_headers': {'date': 'Wed, 10 Dec 2025 11:43:02 GMT', 'content-type': 'application/json', 'content-length': '19', 'connection': 'keep-alive', 'x-pinecone-request-lsn': '1', 'x-pinecone-request-logical-size': '12392', 'x-pinecone-request-latency-ms': '411', 'x-pinecone-request-id': '4670108049108608978', 'x-envoy-upstream-service-time': '238', 'grpc-status': '0', 'server': 'envoy'}})

In [21]:
index.describe_index_stats()

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '181',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 10 Dec 2025 11:43:06 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '4',
                                    'x-pinecone-request-id': '2847433734864407913',
                                    'x-pinecone-request-latency-ms': '3'}},
 'dimension': 768,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'__default__': {'vector_count': 4}},
 'storageFullness': 0.0,
 'total_vector_count': 4,
 'vector_type': 'dense'}

## Similarity Search

In [22]:
query = "technology"
query_embedding = embedding_model.encode(query).tolist()

In [23]:
len(query_embedding)

768

In [24]:
similar_docs = index.query(vector=query_embedding, top_k=2, include_metadata=True)
similar_docs

QueryResponse(matches=[{'id': 'id4',
 'metadata': {'source': 'ai_revolution'},
 'score': 0.218234062,
 'values': []}, {'id': 'id1',
 'metadata': {'source': 'cricket'},
 'score': 0.0997181,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 10 Dec 2025 11:43:18 GMT', 'content-type': 'application/json', 'content-length': '223', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '1', 'x-pinecone-request-latency-ms': '90', 'x-pinecone-request-id': '1281810386957727394', 'x-envoy-upstream-service-time': '3', 'grpc-status': '0', 'server': 'envoy'}})

## CRUD operations on Vector Database

#### Add data

In [25]:
blockchain_news = """
The blockchain industry continues to evolve rapidly, marked by significant technological advancements and regulatory developments.
This month, the spotlight is on the launch of Ethereum 3.0, which promises enhanced scalability and security features.
This upgrade is expected to drastically reduce transaction fees and increase processing speeds, making decentralized applications (dApps) more efficient and user-friendly.
"""

In [26]:
embedding_query = embedding_model.encode(blockchain_news).tolist()

In [27]:
index.describe_index_stats()

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '181',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 10 Dec 2025 11:43:23 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '42',
                                    'x-pinecone-request-id': '4948978432689143878',
                                    'x-pinecone-request-latency-ms': '42'}},
 'dimension': 768,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'__default__': {'vector_count': 4}},
 'storageFullness': 0.0,
 'total_vector_count': 4,
 'vector_type': 'dense'}

In [28]:
index.upsert([{"id":"id5", "values":embedding_query, "metadata":{"source":"blockchain"}}])

UpsertResponse(upserted_count=1, _response_info={'raw_headers': {'date': 'Wed, 10 Dec 2025 11:43:24 GMT', 'content-type': 'application/json', 'content-length': '19', 'connection': 'keep-alive', 'x-pinecone-request-lsn': '2', 'x-pinecone-request-logical-size': '3099', 'x-pinecone-request-latency-ms': '252', 'x-pinecone-request-id': '5167460202032126710', 'x-envoy-upstream-service-time': '166', 'grpc-status': '0', 'server': 'envoy'}})

In [29]:
index.describe_index_stats()

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '181',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 10 Dec 2025 11:43:25 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '3',
                                    'x-pinecone-request-id': '7379918009684965534',
                                    'x-pinecone-request-latency-ms': '3'}},
 'dimension': 768,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'__default__': {'vector_count': 5}},
 'storageFullness': 0.0,
 'total_vector_count': 5,
 'vector_type': 'dense'}

In [30]:
query_embedding = embedding_model.encode("technology").tolist()
similar_docs = index.query(vector=query_embedding, top_k=2, include_metadata=True)
similar_docs

QueryResponse(matches=[{'id': 'id4',
 'metadata': {'source': 'ai_revolution'},
 'score': 0.218234062,
 'values': []}, {'id': 'id5',
 'metadata': {'source': 'blockchain'},
 'score': 0.133591652,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 10 Dec 2025 11:43:26 GMT', 'content-type': 'application/json', 'content-length': '228', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '2', 'x-pinecone-request-latency-ms': '121', 'x-pinecone-request-id': '3804650279324994281', 'x-envoy-upstream-service-time': '35', 'grpc-status': '0', 'server': 'envoy'}})

#### Read data

In [31]:
results = index.fetch(ids=['id1', 'id3'])

In [32]:
for k in results["vectors"]:
  print(results["vectors"][k]['metadata'])

{'source': 'election'}
{'source': 'cricket'}


#### Update data

In [33]:
embedding_query = embedding_model.encode("This is sample document about generative AI").tolist()
index.upsert([("id3", embedding_query, {"source":"gen ai"})])

UpsertResponse(upserted_count=1, _response_info={'raw_headers': {'date': 'Wed, 10 Dec 2025 11:43:31 GMT', 'content-type': 'application/json', 'content-length': '19', 'connection': 'keep-alive', 'x-pinecone-request-lsn': '3', 'x-pinecone-request-logical-size': '3095', 'x-pinecone-request-latency-ms': '246', 'x-pinecone-request-id': '4228916381027592279', 'x-envoy-upstream-service-time': '160', 'grpc-status': '0', 'server': 'envoy'}})

In [34]:
index.fetch(ids=['id3'])

FetchResponse(namespace='', vectors={'id3': Vector(id='id3', values=[0.0174305327, -0.0290010683, -0.0335818306, -0.000871601922, -0.042215541, 0.0295835342, 0.0745305195, -0.00894235168, -0.0314410478, 0.0250650588, 0.0490644723, -0.0251699109, 0.022712402, 0.0440000631, 0.0629395, -0.0993508771, 0.0334789827, 0.0161244776, -0.0355880745, -0.0431858115, -0.0385033563, -0.0146136079, -0.0304313432, 0.00726123, -0.0263031162, -0.0365769044, -0.024414571, -0.0265678987, -0.0156977233, -0.0342872851, -0.00794321671, -0.0204919148, 0.0215113219, -0.0140419621, 1.3077323e-06, -0.0543110818, -0.0130754625, 0.0227110181, 0.027217485, 6.31651e-06, 0.0371696316, 0.0664914325, -0.0291836374, 0.0218777042, -0.0346797034, -0.016341975, 0.0787371174, 0.0665135086, 0.00341358, 0.0438310318, -0.00342690223, -0.048855897, 0.0721274242, -0.0242682789, 0.0627312809, -0.027284842, 0.00072479347, 0.0195972286, 0.0115621854, -0.0264709294, -0.0133053381, 0.00195603864, 0.0237448569, -0.0370910056, 0.030897

#### Delete data

In [35]:
index.fetch(ids=['id2'])

FetchResponse(namespace='', vectors={'id2': Vector(id='id2', values=[-0.00384662021, -0.072715193, -0.0028414533, 0.0574586503, -0.00515252585, -0.0170758776, -0.10618224, -0.0312843956, -0.0369069651, -0.0311232638, -0.0203364436, 0.00393039174, 0.00282162614, -0.0322650373, 0.0612160601, -0.0162303261, -0.00738591375, -0.0254140906, -0.0270358138, -0.0188592896, 0.026998058, 0.023973614, 0.0206892602, -0.00475603715, 0.0217966624, -0.0107850842, -0.00940123852, 0.0341491364, 0.0194722563, -0.0329037569, -0.00297041587, 0.00035784347, 0.0314517505, 4.50246735e-05, 2.13730755e-06, -0.0602920763, -0.0257892758, -0.0438521281, 0.0148153491, -0.0247502867, 0.0146588823, -0.0180866234, -0.0136758564, -0.0262692682, 0.0120234732, 0.0341221578, -0.0325539447, 0.0087031154, -0.00571523793, 0.0301483702, 0.011243714, -0.0319951288, -0.0130925672, -0.00541230338, -0.053039521, -0.0510497, 0.0548728742, -0.0158240404, -0.0467207842, 0.0122886263, 0.0113953333, -0.0501406528, -0.0139560299, -0.00

In [36]:
index.delete(ids=['id2'])

{'_response_info': {'raw_headers': {'date': 'Wed, 10 Dec 2025 11:43:35 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-pinecone-request-lsn': '4',
   'x-pinecone-request-logical-size': '0',
   'x-pinecone-request-latency-ms': '159',
   'x-pinecone-request-id': '2814795796184172005',
   'x-envoy-upstream-service-time': '159',
   'grpc-status': '0',
   'server': 'envoy'}}}

In [37]:
index.fetch(ids=['id2'])

FetchResponse(namespace='', vectors={}, usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 10 Dec 2025 11:43:36 GMT', 'content-type': 'application/json', 'content-length': '53', 'connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1', 'x-pinecone-request-id': '386980109123053009', 'x-envoy-upstream-service-time': '1', 'grpc-status': '0', 'server': 'envoy'}})

## Delete Pinecone Index

In [38]:
pc.delete_index('example-index')